In [2]:
# import dependencies
import pandas as pd
import numerapi
import sklearn.linear_model
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor,  Pool
from tqdm import tqdm
import lightgbm
import numpy as np
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import math
from skimage.transform import resize
from skimage.io import imsave
from skimage.io import imread

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.config.list_physical_devices('GPU')

/home/angel/Desktop/git/numerai/venv/lib/python3.7/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# download the latest training dataset (takes around 30s)
training_data = pd.read_csv("train_data-kaz-10.csv")
# training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
del training_data['Unnamed: 0']
training_data.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,target,era
0,0.625115,0.446158,0.531525,0.335567,0.405357,0.518093,0.474484,0.491956,0.533080,0.486160,...,0.370507,0.552685,0.304854,0.200940,0.481556,0.530445,0.458085,0.519534,0.50,era1
1,0.367690,0.602062,0.500317,0.540271,0.287590,0.489212,0.525986,0.669204,0.377609,0.361885,...,0.267438,0.493574,0.297666,0.407217,0.533826,0.673376,0.336869,0.467933,0.25,era1
2,0.675877,0.360629,0.376649,0.329879,0.526551,0.417776,0.485493,0.662721,0.333822,0.394766,...,0.579851,0.511115,0.387974,0.571073,0.461804,0.673221,0.186182,0.436037,0.25,era1
3,0.644683,0.497132,0.577763,0.658211,0.315884,0.563425,0.812878,0.532486,0.278397,0.444438,...,0.383529,0.600189,0.432368,0.386079,0.595988,0.669761,0.265909,0.360806,0.25,era1
4,0.255260,0.368295,0.403755,0.496896,0.238508,0.562416,0.486466,0.474467,0.118016,0.508690,...,0.605335,0.573041,0.607115,0.321210,0.479349,0.574421,0.481902,0.723485,0.75,era1


In [4]:
# download the latest tournament dataset (takes around 30s)
tournament_data = pd.read_csv("tournament_data-kaz-10.csv")
# tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
del tournament_data['Unnamed: 0']
tournament_data.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,target,era,type,id
0,0.684969,0.537315,0.644790,0.475414,0.522303,0.561284,0.441950,0.583328,0.357338,0.583919,...,0.652358,0.747293,0.560961,0.417274,0.602620,0.593732,0.25,era121,validation,n0003aa52cab36c2
1,0.462252,0.545916,0.394655,0.378866,0.569654,0.708049,0.495076,0.426700,0.565530,0.217472,...,0.594471,0.464464,0.739603,0.395773,0.587933,0.522045,0.50,era121,validation,n000920ed083903f
2,0.582050,0.557493,0.535555,0.323159,0.506384,0.605278,0.365147,0.181733,0.332626,0.691691,...,0.639979,0.540645,0.414977,0.393736,0.625929,0.519788,1.00,era121,validation,n0038e640522c4a6
3,0.419429,0.462896,0.448324,0.513063,0.390499,0.412967,0.390469,0.337310,0.549885,0.500408,...,0.414323,0.467435,0.382670,0.339632,0.606715,0.264071,0.50,era121,validation,n004ac94a87dc54b
4,0.632081,0.405627,0.395501,0.304732,0.428091,0.342243,0.480964,0.703205,0.334977,0.623257,...,0.541503,0.377432,0.390525,0.393758,0.299309,0.440200,0.75,era121,validation,n0052fe97ea0c05f


In [5]:
feature_cols = tournament_data.columns[[x for x in range(50)]]
feature_cols

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49'],
      dtype='object')

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from collections import deque
import numpy as np
import random
import os
import time
import numpy as np  # for array stuff and random
from PIL import Image  # for creating visual of our env
import cv2  # for showing our visual live
import matplotlib.pyplot as plt  # for graphing our mean rewards over time
import pickle  # to save/load Q-Tables
from matplotlib import style  # to make pretty charts because it matters.
import time  # using this to keep track of our saved Q-Tables.
from tqdm import tqdm
style.use("ggplot")  # setting our style!

In [7]:

#...

# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)

In [8]:
training_data[training_data['era']=='era2']

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,target,era
2408,0.486658,0.486152,0.662451,0.357979,0.534530,0.495499,0.356730,0.404762,0.736316,0.415910,...,0.265002,0.681017,0.257958,0.430325,0.543634,0.276219,0.760015,0.430206,0.50,era2
2409,0.624974,0.339195,0.341060,0.434427,0.397542,0.565924,0.451867,0.459982,0.510805,0.614807,...,0.289320,0.321779,0.229865,0.610209,0.414913,0.411060,0.423278,0.421295,0.25,era2
2410,0.653105,0.658849,0.559097,0.461920,0.329984,0.577726,0.411187,0.381742,0.578934,0.489683,...,0.504109,0.394632,0.380040,0.227132,0.477765,0.692810,0.496145,0.456073,0.75,era2
2411,0.575518,0.326938,0.409218,0.332895,0.378200,0.615375,0.453761,0.282744,0.628355,0.426157,...,0.359470,0.474403,0.151060,0.206681,0.382519,0.406021,0.319079,0.371567,0.50,era2
2412,0.530346,0.353012,0.551376,0.339486,0.390872,0.480216,0.605556,0.376597,0.693080,0.674294,...,0.194202,0.364407,0.438383,0.142026,0.408361,0.341363,0.356214,0.474564,0.50,era2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,0.449920,0.550604,0.432674,0.363428,0.644586,0.470710,0.490673,0.587163,0.418448,0.466109,...,0.421833,0.429803,0.242451,0.276027,0.442829,0.571368,0.474072,0.420833,0.25,era2
4775,0.457116,0.568942,0.528643,0.614990,0.599927,0.580172,0.587487,0.436227,0.661158,0.481465,...,0.301094,0.477128,0.450106,0.648919,0.579889,0.391388,0.277539,0.602677,0.00,era2
4776,0.442936,0.708140,0.391498,0.592014,0.317721,0.557839,0.576364,0.709167,0.656288,0.573535,...,0.386059,0.180151,0.509054,0.483241,0.450842,0.541876,0.498826,0.499239,1.00,era2
4777,0.457946,0.423380,0.547968,0.407668,0.373997,0.458044,0.797467,0.543064,0.685613,0.568498,...,0.265693,0.590528,0.305824,0.319498,0.559978,0.460262,0.214847,0.677740,0.75,era2


In [9]:
training_data[training_data['era']=='era2'][feature_cols].iloc[0].values

array([0.486658  , 0.4861524 , 0.662451  , 0.35797867, 0.53452986,
       0.495499  , 0.35672954, 0.40476236, 0.7363155 , 0.4159099 ,
       0.59522957, 0.4701058 , 0.69101566, 0.59315336, 0.4666693 ,
       0.35831574, 0.11075862, 0.42406616, 0.3535127 , 0.5713289 ,
       0.4265387 , 0.51979077, 0.45210803, 0.54398507, 0.38161808,
       0.3488925 , 0.38966388, 0.40048686, 0.6690425 , 0.3473469 ,
       0.48347417, 0.35764343, 0.4173858 , 0.6222329 , 0.4851353 ,
       0.21497218, 0.43060246, 0.4019114 , 0.5263327 , 0.5893069 ,
       0.47857782, 0.46592584, 0.26500192, 0.6810171 , 0.25795817,
       0.43032524, 0.5436337 , 0.27621865, 0.76001495, 0.43020645])

In [10]:
indicator_predict = np.ones((50))*-1            
allprediction = [el for el in indicator_predict]
allprediction.append(0.1)

In [11]:
a=0
allprediction[-a1:]

NameError: name 'a1' is not defined

In [113]:
def correlation_score(y_true, y_pred):
        return np.corrcoef(y_true, y_pred)[0,1]
    
class Enviroment:
    
    def __init__(self, data, features):
        # len_obs_space = totalFeatures + 50 forward correlation        
        self.extrafeatures = 100
        self.OBSERVATION_SPACE_VALUES = len(features) +  self.extrafeatures + 1
        # cada accion seran en steps de 0.01
        self.ACTION_SPACE_SIZE = 100
        self.data = data
        self.features = features
        self.unique_eras = data.era.unique()
        
        
    def reset(self):
        
        self.currentEra = np.random.choice(self.unique_eras, 1)[0]
        
        self.currentPoint = 0
        self.len_stage = len(self.data[self.data['era']==self.currentEra])
        indicators = self.data[self.data['era']==self.currentEra][self.features].iloc[self.currentPoint].values
        indicator_predict = np.ones((self.extrafeatures))*-1            
        self.allprediction = [el for el in indicator_predict]
        self.allcorrelations = [el for el in indicator_predict]
        self.incremntalStep = [0]
        self.currentCorrelation = 1
        
        return np.concatenate((indicators, indicator_predict, self.incremntalStep ))
    
    def step(self, action):
        
        newAction = action / 100
        # if action == 0:
#             newAction= 0.
#         elif action == 1:
#             newAction = 0.25
#         elif action == 2:
#             newAction = 0.5
#         elif action == 3:            
#             newAction = 0.75                
#         elif action == 4:            
#             newAction = 1.                            
        self.allprediction.append(newAction)
        
        
        if self.currentPoint < self.extrafeatures:
            trueActions = self.data[self.data['era']==self.currentEra]['target'].iloc[0:self.currentPoint+1].values    
            # print(trueActions)
            predActions = np.array(self.allprediction[-self.currentPoint-1:])
            # print(predActions)
            newCorrelation = correlation_score(predActions, trueActions )
        else:
            trueActions = self.data[self.data['era']==self.currentEra]['target'].iloc[0:self.currentPoint+1].values    
            newCorrelation = correlation_score(np.array(self.allprediction[-len(trueActions):]), trueActions )

        
        if not self.currentPoint % 300:            
            print(newCorrelation)
            
        if np.isnan(newCorrelation):
            newCorrelation=0
# #            reward = 0
#        else:
#            reward = newCorrelation
            
        self.allcorrelations.append(newCorrelation)
        
        # reward = -abs(newCorrelation-self.currentCorrelation)
        
        if newCorrelation<self.currentCorrelation:
            reward = -abs(newCorrelation-self.currentCorrelation)
        else:
            reward = abs(newCorrelation-self.currentCorrelation)
        
        self.currentCorrelation = newCorrelation
        
        done = True if self.currentPoint+1 >= self.len_stage else False
        
        
        if done:
            print("oldCorrelation", self.currentCorrelation)
            reward = self.currentCorrelation
        
        indicators = self.data[self.data['era']==self.currentEra][self.features].iloc[self.currentPoint].values
        indicators_predict = np.array(self.allprediction[-self.extrafeatures:])
        new_state = np.concatenate((indicators, indicators_predict, [self.currentPoint/self.len_stage]))
        
        self.currentPoint += 1
        return new_state, reward, done 

In [114]:
env = Enviroment(training_data, feature_cols)

env.reset()

array([ 0.48521546,  0.36940157,  0.4270708 ,  0.6455298 ,  0.21197662,
        0.71584696,  0.3949961 ,  0.34914654,  0.2798578 ,  0.6336333 ,
        0.42224094,  0.5082413 ,  0.47696763,  0.36014792,  0.44837514,
        0.18299386,  0.38406605,  0.5978968 ,  0.51530063,  0.31999657,
        0.51492155,  0.5217685 ,  0.59950227,  0.5969795 ,  0.5051454 ,
        0.28242147,  0.4085625 ,  0.27547157,  0.43815252,  0.6439412 ,
        0.47847757,  0.22753529,  0.54245603,  0.48251635,  0.37717324,
        0.31087372,  0.39671618,  0.6786706 ,  0.60215396,  0.64564544,
        0.5549608 ,  0.31358397,  0.34460256,  0.64929837,  0.23681913,
        0.29293802,  0.32200834,  0.39856413,  0.5839887 ,  0.44158044,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.  

In [115]:
env.step(2)

nan


(array([ 0.48521546,  0.36940157,  0.4270708 ,  0.6455298 ,  0.21197662,
         0.71584696,  0.3949961 ,  0.34914654,  0.2798578 ,  0.6336333 ,
         0.42224094,  0.5082413 ,  0.47696763,  0.36014792,  0.44837514,
         0.18299386,  0.38406605,  0.5978968 ,  0.51530063,  0.31999657,
         0.51492155,  0.5217685 ,  0.59950227,  0.5969795 ,  0.5051454 ,
         0.28242147,  0.4085625 ,  0.27547157,  0.43815252,  0.6439412 ,
         0.47847757,  0.22753529,  0.54245603,  0.48251635,  0.37717324,
         0.31087372,  0.39671618,  0.6786706 ,  0.60215396,  0.64564544,
         0.5549608 ,  0.31358397,  0.34460256,  0.64929837,  0.23681913,
         0.29293802,  0.32200834,  0.39856413,  0.5839887 ,  0.44158044,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1. 

In [116]:
while not done:
    _,_, done = env.step(0.3)

0.06611742602926432
0.04637645073977602
0.03824019859422745
0.03193491510352962
0.02843795879366878
0.026099745377254274
0.024219531333018025


KeyboardInterrupt: 

In [ ]:
a = np.array([0.5,  0.75, 0.25, 0.5,  0.5,  0. ,  0.  ])
b = np.array([0.001, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002])
correlation_score(a, b)

In [ ]:
if not os.path.isdir('models'):
    os.makedirs('models')

In [ ]:
e=1
d=0.85


for x in range(4):
    e*=d

print(e)

In [117]:

DISCOUNT = 0.99

REPLAY_MEMORY_SIZE = 150_000
MIN_REPLAY_MEMORY_SIZE = 1_000

MODEL_NAME = "256x2"
MINIBATCH_SIZE = 64
UPDATE_TARGET_EVERY = 5


EPISODES = 20_000
MIN_REWARD = -200
AGGREGATE_STATS_EVERY = 10


epsilon = 0.8  # not a constant, going to be decayed
EPSILON_DECAY = 0.85
MIN_EPSILON = 0.001

ep_rewards = []
aggr_ep_rewards = {'ep': [], 'avg': [], 'max': [], 'min': []}


class DQNAgent:
    def __init__(self):
        # our main model this going to train every step
        self.model = self.create_model()
        
        # target model this is what predict again every step
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())
        
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        
        self.tensorboard = ModifiedTensorBoard(log_dir=f"logs/{MODEL_NAME}{int(time.time())}")
        
        self.target_update_counter = 0
        
    
    def create_model(self):
        model = Sequential()

        model.add(Dense(256, input_shape=(env.OBSERVATION_SPACE_VALUES,)))  # OBSERVATION_SPACE_VALUES = (10, 10, 3) a 10x10 RGB image.
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu')) 
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu')) 
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu')) 
        model.add(Dropout(0.2))
        

        model.add(Dense(env.ACTION_SPACE_SIZE, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=['accuracy'])
        return model
    
    def create_model_2(self):
        model = Sequential()

        model.add(Conv2D(256, (3, 3), input_shape=env.OBSERVATION_SPACE_VALUES))  # OBSERVATION_SPACE_VALUES = (10, 10, 3) a 10x10 RGB image.
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        model.add(Conv2D(256, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
        model.add(Dense(64))

        model.add(Dense(env.ACTION_SPACE_SIZE, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=['accuracy'])
        return model
    
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/1)[0]
    
    def train(self, terminal_state, step):
        if len(self.replay_memory) <  MIN_REPLAY_MEMORY_SIZE:
            return
        
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        
        current_states = np.array([transition[0] for transition in minibatch])/1
        current_qs_list = self.model.predict(current_states)
        
        new_current_states = np.array([transition[3] for transition in minibatch])/1
        
        future_qs_list = self.target_model.predict(new_current_states)
        
        X = []
        y = []
        
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward
                
            current_qs = current_qs_list[index]
            current_qs[action] = new_q
            X.append(current_state)
            y.append(current_qs)
        
        # print("fiting")
        self.model.fit(np.array(X)/1, np.array(y), batch_size=MINIBATCH_SIZE, 
                       verbose = 0, shuffle=False)
                       # callbacks =[self.tensorboard] if terminal_state else None )
        
        if terminal_state:
            self.target_update_counter += 1
     
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0
            

In [118]:
agent = DQNAgent()

for episode in tqdm(range(1, EPISODES+1), ascii=True, unit="episode"):
    agent.tensorboard.step = episode
    episode_reward = 0
    step = 1
    current_state = env.reset() 
    done = False
    print("NEW ENV")
    while not done:
        if np.random.random() > epsilon:
            action = np.argmax(agent.get_qs(current_state))
        else:
            action = np.random.randint(0, env.ACTION_SPACE_SIZE)
            
        new_state, reward, done = env.step(action)
        
        episode_reward += reward
        
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)
        current_state = new_state
        step += 1
        
    ep_rewards.append(episode_reward)
    
    if not episode % AGGREGATE_STATS_EVERY or episode ==1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        # agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)
        if min_reward >=MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')
    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

  0%|          | 0/20000 [00:00<?, ?episode/s]

NEW ENV
nan
0.011042043859800434
-0.01148162343154989
-0.011723114463510728
-0.01964932826836501
-0.01883209107116685
-0.017598666965306733
-0.016632175780159464
-0.004089942423016557
0.003467267155481871
-0.0005621879668908539
-0.005851951316802512
-0.018392024113182456
-0.024447308457132746
-0.019404535545152184
-0.016293328222537054
oldCorrelation -0.01642864047828511
INFO:tensorflow:Assets written to: models/256x2____-1.03max___-1.03avg___-1.03min__1605812733.model/assets


  0%|          | 1/20000 [07:38<2549:50:27, 458.99s/episode]

NEW ENV
nan
-0.03825344746658556
-0.06804770535409249
-0.03468203646060476
0.013539008677427068
0.008416790557026738
0.010569042732062576
0.01328426515691432
0.014140665828589178
0.011772298026606693
0.013649481871823304
0.015331483512198136
0.022373816364904965
0.013822050097658658
0.018031337388442686
0.018301286251540913


  0%|          | 2/20000 [16:52<2707:05:18, 487.32s/episode]

...

oldCorrelation -0.005295708036799557
NEW ENV
nan
-0.08970887896883405
-0.038128675682927216
-0.03559725396729917
-0.04060040292321137
-0.03672537782929435
-0.04714736903566513
-0.048380525390845815
-0.03816111789038159
-0.04167173216264202
-0.03425392785538007
-0.02703908397657494
-0.024110363242047007
-0.02378865370698699


  1%|          | 128/20000 [19:14:25<2921:57:26, 529.34s/episode]

oldCorrelation -0.022252238609442487
NEW ENV
nan
0.009294148159607753
-0.008594943705292866
-0.01704972922390848
-0.0016347861095768871
0.0167706407826799
0.010749857416436954
-0.0005830341602870853


  1%|          | 128/20000 [19:19:56<3001:22:10, 543.73s/episode]


KeyboardInterrupt: 